# Miniature/spotaneous postsynaptic currents
Analyzing miniature/spontaneous postsynaptic currents (m/sPSCs) is one of the most common analyses in patch clamp electrophysiology. m/sPSCs are ionic currents from AMPA, NMDA, glycine, or GABAA receptors that are evoked due to release of a single or multiple synaptic vesicles. We will cover both mPSCs and sPSCs, and go over how to find and analyze PSC events. While this chapter is PSCs, most of the theory applies to miniature/spontaneous postsynaptic potentials (PSPs). There are several experiments you can do; synapse number, silent synapses, excitatory/inhibitory ratio, synaptic multiplicity, and changes in synaptic release regulated by other non-ionic receptors.

## Miniature postsynaptic currents
Miniature postsynaptic currents (mPSCs) are ionic currents evoked from the release of a single synaptic vesicle {cite}`del_castillo_quantal_1954`. Frequency of mPSCs is used as a proxy for the number of functional synapses (synapses that have presynaptic input) that contain the receptor of interest (but not necessarily the number of synapses). The interpretation of mPSC data depends on what receptor you are recording from. If you are recording mEPSCs from AMPARs then you are likely getting the number of "active" or "non-silent" synapses. If you are recording NMDARs could be looking at the number of silent synapses (only if you compare to the AMPAR mEPSCs). If you are recording mIPSCs then you are getting the number of inhibitory synapses. With mIPSCs you could be getting GABAAR or GlyR. One important caveat of mPSCs is that you are not getting where the presynaptic input is coming from. If you want projection specific synaptic input you need to run a different type of experiment than we will be covering here, but is covered later in the book.

### Internal and external solutions
You will need to block spontaneous activity including tetrodotoxin (TTX) in the bath. Preferably you would also use an internal solution that contains cesium and QX-314. For more information on internals see {doc}`./internal_solutions.ipynb`. Depending on the receptor current(s) you want to record you will need to block certain other receptors by including specific drugs in the external solutions. For more information on externals see {doc}`./external_solutions.ipynb`.

### How should you record mEPSCs
There are two primary ways you can record mEPSCs. One way is you can record continuously for about 3-5 minutes. Technically speaking this is the easiest method since you have a single recording and pretty much any simple recording software will implement this method. The second way is you can record 20-40 sweeps/acquisitions of 5-15 seconds each. This method allows you to discard bad portions of the recording. Usually when you get proficient at patching you will rarely have bad recordings however sometimes you get 30 seconds where there is an unstable seal, digital cell phone noise, your bath gets too low, you get pump/vacuum noise or other issues. When this happens it is fine to discard the 5 or so acquisitions that are bad. You can create acquisitions from continuous recordings by splitting to get the same benefits of the sweeps/acquisitions method.

## Analyzing miniature/spontaneous PSCs.
There are several important features of m/aPSCs that you will want to analyze. The primary feature is frequency (the number of events occuring every second). Frequency is used as a proxy for the number of synapses that contain the receptor whose currents you are recording. The more mPSCs, the more synapses. However, if there are changes in release probability you could also get a change in frequency without a change in synapse number. To determine whether there are changes in release probability you will need to run some pair-pulse experiments which are described in a later chapter. Another feature is mPSC amplitude. A larger amplitude could mean two things. Larger mPSCs could mean there are more receptors at the postsynaptic element. Alternatively, larger mPSCs could mean that you get less distal synapses due to decreased dendritic length or more synapses close to the cell body (probably less likely to occur). Amplitude of mPSCs decreases the further from the cell body that the event occurs due to dendritic filtering. To rule out one of the interpretations you can use the mEPSC rise rate (amplitude/peak_time-baseline_start). More distal mPSCs should have a slower rise rate due to dendritic lowpass filtering. Lastly, you can look at the tau or decay rate of the mPSC. Changes in tau are usually due to changes in receptor subunit composition. Tau is especially useful when you need to identify specific cells types. Interneurons, like parvalbumin interneurons, have a very short mPESC tau compared to pyramidal neurons. Tau can also be affected by dendritic filtering.